In [1]:
import csv
import random
import os
import glob

import pandas as pd
import numpy as np
import math
import cv2
import skimage.external.tifffile as tiff

from common import Statistics, dataset_source
from pathlib import Path

from resources.conv_learner import *

from tqdm import tqdm_notebook as tqdm

In [2]:
path = "datasets/HPA_challenge_2018/"
# path = "datasets/Kagg"

# Dataset generation

Below are two functions which: 

a) select from the original csv a certain percentage of images at random and save them in a second csv<br/>
b) based on an input csv, select image files, compose stacks of images according to selected channels and save them in a separate folder as tiff files.

## Generate train subset

In [5]:
def generate_random_subset_csv(path, infile, outfile, percentage = 10, seed=None):

    perc = math.ceil((int(100 / percentage)))
    if seed: random.seed = seed # sharing outfiles directly might be easier
    rand_subset = []
    print(f"Generating random {perc}% subset of {infile}...")
    
    with open(path + infile, 'r') as d:
        reader = csv.reader(d)
        train_data = np.vstack(list(reader))

    for idx, file in enumerate(train_data[:,0]):
        number = random.randint(1,perc)
        if number == 1:
            rand_subset.append(train_data[idx])
    
    random_subset = np.vstack(rand_subset)
    
    # write to csv in the same directory:
    # some error-catching might be nice if outfile already exists
    df = pd.DataFrame(random_subset)    
    df.to_csv(path + outfile, header = False, index = False)
    print(f"Success: saved random {perc}% subset to {outfile}")
    
    return random_subset

In [6]:
# generate random subset csv file:

random_ten_percent = generate_random_subset_csv(path, 'train.csv', 'train_10perc_v2.csv', seed = 1)
# print(random_ten_percent[:10])

Generating random 10% subset of train.csv...
Success: saved random 10% subset to train_10perc_v2.csv


In [ ]:
# Check for consistency between subset and source

a = list(train_data[:,0])
b = list(random_ten_percent[:,0])

c = set(a) & set(b)
assert len(b) == len(c), "Selected images not subset of original data"
print('Check successful')

## Generate dataset 

dataset_gen():

1) Generates composite images (if multiple channels have been selected) from the individual blue, green, yellow and red channels of the original images.<br/>
2) The input csv defines the exact images to be selected.<br/>
3) Images are stored in a newly generated folder. 

In [38]:
def dataset_gen(csv_path, source_path, output_path, channels = ['green']):
    
    with open(csv_path, 'r') as d:
        reader = csv.reader(d)
        fnames = np.vstack(list(reader))[1:,0] #### change depending on header ###
    
    assert not os.path.isdir(output_path), 'Chosen output directory already exists!'
    print(f"creating output directory: {os.path.basename(output_path)}")
    os.mkdir(output_path)
    
    for f in tqdm(fnames, total = len(fnames), unit="files"):
        im_ = [cv2.imread(str(source_path + f + '-' + c + '.png'), cv2.IMREAD_GRAYSCALE) for c in channels]
        im = np.stack(im_)
        # im = np.rollaxis(im,0,3)
        
        assert im.dtype == 'uint8' # making sure files are expected dtype; may change for different datasets...
        tiff.imsave(output_path + f + '.tiff', im, imagej = True) # imageJ = True generates ImageJ compatible Hyperstack

In [26]:
len(fnames)

74606

In [39]:
### - use generated subset csv to generate folder with tiff files composed of select channels

path = "datasets/HPA_challenge_2018/"

csv_path = path + 'HPAv18_60x_def_dupes_removed.csv'
source_path = path + 'HPAv18_BGRY_source/'
output_path = path + 'HPAv18_GRBY_60x_def_dupes_removed_all/'
channels = ['green','red','blue', 'yellow'] # can also leave this out since default is 'green'

dataset_gen(csv_path, source_path, output_path, channels = channels)

creating output directory: 


In [18]:
!ls datasets/HPA_challenge_2018/HPAv18_BGRY_source/ -1 | wc -l

/bin/sh: 1: ls: Argument list too long
0


In [37]:
!ls datasets/HPA_challenge_2018/

HPA_labels.csv			     models
HPAv18RBGY_wodpl.csv		     needles.csv
HPAv18_60x_def_dupes_removed.csv     sample_submission.csv
HPAv18_BGRY_source		     test-raw
HPAv18_BGR_all			     test.zip
HPAv18_BGR_source		     test_BGR_all
HPAv18_def_dupes_removed.csv	     test_BGYR_all
HPAv18_dupes_400.csv		     tmp
HPAv18_dupes_5k.csv		     train.csv
HPAv18_dupes_no_si_dist5.csv	     train.zip
HPAv18_dupes_no_si_dist5.pkl	     train_BGR_all
HPAv18_dupes_no_si_phash_10_BGR.pkl  train_BGYR_all
HPAv18_wodpl_60x.csv		     train_raw
haystack_no_si.csv


In [31]:
d = os.listdir('datasets/HPA_challenge_2018/HPAv18_BGR_all')
tiff.imread('datasets/HPA_challenge_2018/HPAv18_BGR_all/' + d[0]).shape

(3, 512, 512)

In [19]:
!rm -rf datasets/HPA_challenge_2018/HPAv18_BGR_all

In [4]:
# create non-header version of input .csv

# csv_path = path + 'HPA_labels.csv'


# with open(csv_path, 'r') as d:
#     reader = csv.reader(d)
#     fnames = np.vstack(list(reader))
    
# no_header = fnames[1:] # remove header line

# no_header_df = pd.DataFrame(no_header)
# no_header_df

# no_header_df.to_csv(path + 'HPA_labels.csv', header = False, index = False)

## Compiling test-set images

In [7]:
def testset_gen(path, test_folder, output_path, channels = ['green']):
    
    _fnames = test_names = read_dir(path, test_folder)
    _fnames = pd.DataFrame([os.path.basename(_fnames[i].split('_')[0]) for i, _ in enumerate(_fnames)])
    fnames = _fnames[0].unique();
    
    assert not os.path.isdir(output_path), 'Chosen output directory already exists!'
    print(f"creating output directory: {os.path.basename(output_path)}")
    os.mkdir(output_path)
    
    for f in tqdm(fnames, total = len(fnames), unit="files"):
        im_ = [cv2.imread(str(path + test_folder + f + '_' + c + '.png'), cv2.IMREAD_GRAYSCALE) for c in channels]
        im = np.stack(im_)
        # im = np.rollaxis(im,0,3)

        assert im.dtype == 'uint8' # making sure files are expected dtype; may change for different datasets...
        tiff.imsave(output_path + f + '.tiff', im, imagej = True) # imageJ = True generates ImageJ compatible Hyperstack

In [8]:
### - use generated subset csv to generate folder with tiff files composed of select channels

path = path
test_folder = 'test-raw/'
output_path = path + 'test_BGR_all/'
channels = ['blue','green', 'red'] # can also leave this out since default is 'green'

testset_gen(path, test_folder, output_path, channels = channels)

creating output directory: 


# Resources WIP/Code storage:

In [9]:
yeast = tiff.imread(str('datasets/yeast_v11.1/train/02_WT/WT_WP_E2_Mito0_S2_F1_I1_C1_A0.tifstack.tif'))
yeast.shape

(2, 200, 200)

In [9]:
# """ImageJ=1.52b
# images=2
# slices=2
# unit=micron
# spacing=0.3
# loop=false
# min=0.0
# max=65535.0"""